<a href="https://colab.research.google.com/github/JCaballerot/Estadistica_basica_RProject/blob/main/Laboratorio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importamos la data de precio de inmuebles 
library(readr)
df <- read.csv("house_price_data.csv")
View(df)

In [ ]:

# Cálculo de variables de antiguedad
help(substr)

df$antiguedad = as.numeric(substr(df$date, 1, 4)) - df$yr_built


##1. Medidas de tendencia central


In [ ]:
### Análisis de medias

# Veamos cuales son los tres estados con mayor precio promedio de inmuebles
mean(df$price)
dfRes = aggregate(df$price, list(df$city), FUN=mean) 
dfRes = dfRes[order(dfRes$x, decreasing=TRUE), ]
head(dfRes, 3)
# ¿cuáles son los tres estados con menor precio promedio de inmuebles?


In [ ]:
### Análisis de medianas

# Veamos cuales son los tres estados con mayor mediana de precio de inmuebles
median(df$price)
dfRes = aggregate(df$price, list(df$city), FUN=median) 
dfRes = dfRes[order(dfRes$x, decreasing=TRUE), ]
head(dfRes, 3)
# ¿Los resultados coinciden?, ¿A qué podrían deverse las diferencias?

In [ ]:

### Análicemos los outliers en la variable

# Boxplot
boxplot(df$price, horizontal = TRUE)
# ¿La variable presenta outliers?, ¿Qué tipo de tratamiento podríamos aplicar?

install.packages("dplyr")  
library(dplyr)             
df_filtered <- filter(df, df$city %in% c("Medina", "Yarrow Point", "Clyde Hill"))
boxplot(price ~ city, data = df_filtered)

In [ ]:
### Tratamiento de outliers
Q1 <- quantile(df$price, c(0.25))
Q3 <- quantile(df$price, c(0.75))

replace_outliers <- function(x, removeNA = TRUE){
  qrts <- quantile(x, probs = c(0.25, 0.75), na.rm = removeNA)
  iqr <- qrts[2]-qrts[1]
  h <- 1.5 * iqr
  x[x<qrts[1]-h] <- qrts[1]-h
  x[x>qrts[2]+h] <- qrts[2]+h
  x}

df$price_treat <- replace_outliers(df$price)
boxplot(df$price_treat, horizontal = TRUE)
summary(df)

##2. Medidas de variabilidad


In [ ]:
### Histograma Simple 
hist(df$price_treat)

In [ ]:
### Desviación estándar
sd(df$price_treat)

mean(df$price_treat) - sd(df$price_treat)
mean(df$price_treat) + sd(df$price_treat)

# ¿Que variable presenta mayor variabilidad, el número de baños o el número de pisos?
sd(df$bathrooms)
sd(df$sqft_living)

In [ ]:
### coeficiente de variación
cv_bathrooms = sd(df$bathrooms) / abs(mean(df$bathrooms))
cv_sqft_living    = sd(df$sqft_living) / abs(mean(df$sqft_living))

print(cv_bathrooms)
print(cv_sqft_living)

# calcule el coeficiente de variación de tres variables más e indique cuál es la que presenta mayor variabilidad


##3. Distribuciones de probabilidad


In [ ]:
hist(df$price_treat)

In [ ]:
### Test de normalidad 
#Test de Shapiro-Wilk

shapiro.test( x = df$price_treat )

# Simulando una muestra con distribución normal
help(rnorm)
sample_norm = rnorm(1000)
hist(sample_norm)

shapiro.test( x = sample_norm )

# realice una prueba de hipótesis para otras variables del dataset, ¿alguna de ellas cumple con el criterio de normalidad?


In [ ]:


### Intervalos de confianza

# intervalo de confianza al 95%
LI <- mean(df$price_treat) - 1.96*sd(df$price_treat)
LS <- mean(df$price_treat) + 1.96*sd(df$price_treat)
sprintf("El intervalo de confianza al 95 de confianza de la variable precio va de %s a %s", LI, LS)

# intervalo de confianza al 99%
LI <- mean(df$price_treat) - 2.52*sd(df$price_treat)
LS <- mean(df$price_treat) + 2.52*sd(df$price_treat)
sprintf("El intervalo de confianza al 99 de confianza de la variable precio va de %s a %s", LI, LS)

# Calcule el intervalo de confianza para dos variables más al 90%, 95% y 99%



##4. Regresión Lineal

In [ ]:

### Simple Scatterplot
help(sample_n)
df_sample = sample_n(df, 500)

plot(df_sample$bedrooms, df_sample$price_treat)
plot(df_sample$bedrooms, df_sample$price_treat, main="Scatterplot",
     xlab="Area del livingt ", ylab="Precio del inmueble")

# Agregar línea
abline(lm(df_sample$price_treat~df_sample$bedrooms), col="red") # regression line (y~x)

# Realice un Scatterplot de tres variables más y señale el tipo de relación entre los datos 


In [ ]:

### Correlación entre variables

cor(df_sample$bedrooms, df_sample$price_treat)
cor(df_sample$antiguedad, df_sample$price_treat)

# Calcule la correlación entre 3 variables más e indique cuál de ellas presenta una mayor relación con el target

# Test de correlación
cor.test(df_sample$bedrooms, df_sample$price_treat)
cor.test(df_sample$antiguedad, df_sample$price_treat)

# Calcule un test de correlación entre 3 variables más y analice los resultados


# Coeficiente de contingencia
table(df$condition, df$view)

help(chisq.test)
chisq.test(df$condition, df$view)
chi <- chisq.test(df$condition, df$view)
coef_conting = sqrt(chi$statistic / (chi$statistic + length(df$condition)))



In [ ]:

### Regresión lineal simple

reg1 <- lm(price_treat ~ bedrooms, data = df)
summary(reg1)

par(mfrow = c(2, 2))
plot(reg1)

# Residual vs Fitted: Deberia estar distribuidos aleatoriamente alrededor de
# la linea horizontal que representa un error residual de cero

# Normal Q-Q: deberia sugerir que los errores residuales se distribuyen
# normalmente.

# Scale-Location Muestra la raiz cuadrada de los residuos estandarizados,
# como una funcion de los valores ajustados. No deberia existir una
# tendencia clara en ese trama.

# Residual vs Leverage Las distancias mas grandes que 1 son sospechosos y
# sugieren la presencia de un valor atipico posible. y su eliminacion podria
# tener efectos sobre la regresion.



In [ ]:

### Regresión lineal múltiple

reg1 <- lm(price_treat ~ bedrooms + antiguedad, data = df)
summary(reg1)

par(mfrow = c(2, 2))
plot(reg1)

plot(df$bedrooms, df$price_treat)
abline(reg1) 

